In [1]:
import json
import os.path

import torch.optim as optim
import torch
from torch.utils.data import DataLoader

from utils import find_last_model, load_data, prepare_data
from dataloader.SwingDatasets import HourlyDataset
from dataloader.SuperTrainerDataset import SuperTrainerDataset
from trainer.trainer import HourlySwingModelTrainer
from model.swing_model_wp import HourlySwingModel

from data.dataset_paths import forex_path_dict, crypto_path_dict, index_path_dict, tz_dict

In [2]:
pair_types_dict = {
    "forex": forex_path_dict,
    "crypto": crypto_path_dict,
    "index": index_path_dict
}

with open("config_pretrain.json", "r") as file:
    config_train = json.load(file)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
with open("config_pretrain.json", "r") as file:
    config_train = json.load(file)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# train cfg
LOAD_MODEL = config_train["load_model"]
TYPE = config_train["type"]
WINDOW_SIZE = config_train["window_size"]
BATCH_SIZE = config_train["batch_size"]
VALUE_ONLY = config_train["value_only"]
LR = config_train["lr"]
WD = config_train["wd"]
EPOCHS = config_train["epochs"]
EVAL_PERIOD = config_train["eval_period"]
CHECKPOINT_PERIOD = config_train["checkpoint_period"]
MODEL_OUT_PATH = config_train["model_out"]

In [4]:
# model cfg
with open("config_model.json", "r") as file:
    config_model = json.load(file)

inp_dim = config_model["inp_dim"]                          # fix
metadata_dim = config_model["metadata_dim"]                # fix
metadata_bias = config_model["metadata_bias"]              
metadata_gate_bias = config_model["metadata_gate_bias"]   
fusion_model_dim = config_model["fusion_model_dim"]        
fusion_num_heads = config_model["fusion_num_heads"]        
fusion_num_layers = config_model["fusion_num_layers"]      
fusion_apply_grn = config_model["fusion_apply_grn"]        
fusion_dropout = config_model["fusion_dropout"]            
max_window = config_model["max_window"]                    # fix
positional_info = config_model["positional_info"]          
lstm_num_layers = config_model["lstm_num_layers"]          
lstm_bidirectional = config_model["lstm_bidirectional"]    
lstm_dropout = config_model["lstm_dropout"]               
loss_punish_cert = config_model["loss_punish_cert"]       

In [5]:
# load model
model = HourlySwingModel(inp_dim=inp_dim, metadata_dim=metadata_dim, metadata_bias=metadata_bias,
                         metadata_gate_bias=metadata_gate_bias, fusion_model_dim=fusion_model_dim,
                         fusion_num_heads=fusion_num_heads, fusion_num_layers=fusion_num_layers,
                         fusion_apply_grn=fusion_apply_grn, fusion_dropout=fusion_dropout,
                         max_window=max_window, positional_info=positional_info,
                         lstm_num_layers=lstm_num_layers, lstm_bidirectional=lstm_bidirectional,
                         lstm_dropout=lstm_dropout, loss_punish_cert=loss_punish_cert, device=device)
model.to(device)
if LOAD_MODEL is not None:
    model_file = find_last_model(LOAD_MODEL)
    print(f"Loading model from: {os.path.join(LOAD_MODEL, model_file)}")
    model.load_state_dict(torch.load(os.path.join(LOAD_MODEL, model_file), map_location=device))

In [6]:
num_learnable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
num_learnable_params

183038

In [7]:
# load data and prepare super_dataset
pair_path_dict = pair_types_dict[TYPE]
super_train_dataset = SuperTrainerDataset()
super_dev_dataset = SuperTrainerDataset()
idx = 0
for pair, path in pair_path_dict.items():
    idx += 1
    print(f"{idx} - Loading {pair.upper()} from {path}")
    # load data
    pair_data = load_data(path, add_zigzag_col=True)
    pair_data = prepare_data(pair_data, tz_dict[pair])
    # prepare dataset
    print(f"Building train dataset...")
    train_dataset = HourlyDataset(pair_data, WINDOW_SIZE, "train")
    print(f"Building dev dataset...")
    dev_dataset = HourlyDataset(pair_data, WINDOW_SIZE, "dev")
    # add to super dataset
    super_train_dataset.add_dataset(train_dataset)
    super_dev_dataset.add_dataset(dev_dataset)
    print(f"{pair.upper()} loaded.\n")

train_dataloader = DataLoader(super_train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                              collate_fn=HourlyDataset.get_collate_fn())
dev_dataloader = DataLoader(super_dev_dataset, batch_size=32, shuffle=False,
                            collate_fn=HourlyDataset.get_collate_fn())

1 - Loading AUDUSD from data/forex/AUDUSD.csv
Building train dataset...
Trying to replace 1 monday data (0) with (1)
Building dev dataset...
Trying to replace 1 monday data (0) with (1)
AUDUSD loaded.

2 - Loading EURUSD from data/forex/EURUSD.csv
Building train dataset...
Building dev dataset...
EURUSD loaded.

3 - Loading GBPUSD from data/forex/GBPUSD.csv
Building train dataset...
Trying to replace 2 monday data (0) with (1)
Building dev dataset...
Trying to replace 2 monday data (0) with (1)
GBPUSD loaded.

4 - Loading NZDUSD from data/forex/NZDUSD.csv
Building train dataset...
Trying to replace 2 monday data (0) with (1)
Building dev dataset...
Trying to replace 2 monday data (0) with (1)
NZDUSD loaded.

5 - Loading USDCAD from data/forex/USDCAD.csv
Building train dataset...
Trying to replace 2 monday data (0) with (1)
Building dev dataset...
Trying to replace 2 monday data (0) with (1)
USDCAD loaded.

6 - Loading USDCHF from data/forex/USDCHF.csv
Building train dataset...
Trying t

In [ ]:
# save the cfg before start
HourlySwingModelTrainer.save_cfg(config_model, config_train, out_path=MODEL_OUT_PATH)

# train for value only
if VALUE_ONLY:
    model.set_value_only(True)
    optimizer_reg = optim.Adam(model.parameters(), lr=LR)
    trainer = HourlySwingModelTrainer(model, train_dataloader, dev_dataloader, None,
                                      optimizer_reg, device)
    trainer.train(epochs=EPOCHS, eval_period=EVAL_PERIOD,
                  checkpoint_period=CHECKPOINT_PERIOD, out_path=os.path.join(MODEL_OUT_PATH, "reg"))

Model configuration saved to out/pretrain/forex/grn-sin/model_config.json
Training configuration saved to out/pretrain/forex/grn-sin/training_config.json


Evaluating: 100%|███████████████| 2355/2355 [05:24<00:00,  7.26it/s, loss=0.451]


Losses saved to out/pretrain/forex/grn-sin/reg/loss_results.txt


Evaluating: 100%|███████████████| 2355/2355 [05:26<00:00,  7.22it/s, loss=0.463]


Losses saved to out/pretrain/forex/grn-sin/reg/loss_results.txt


...

Evaluating: 100%|████████████████| 2355/2355 [05:15<00:00,  7.45it/s, loss=0.48]


Losses saved to out/pretrain/forex/grn-sin/reg/loss_results.txt


Evaluating: 100%|███████████████| 2355/2355 [05:21<00:00,  7.34it/s, loss=0.451]


Losses saved to out/pretrain/forex/grn-sin/reg/loss_results.txt


Epoch 8/100:   3%|▍              | 223/8240 [01:11<42:00,  3.18it/s, loss=0.463]